
$\newcommand{\br}{\mathbf{r}}$
$\newcommand{\brr}{\mathbf{r}}$
$\newcommand{\bs}{\mathbf{s}}$
$\newcommand{\bss}{\mathbf{s}}$

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
display(HTML("<style>.container { width:80% !important; }</style>"))

# Metadynamics Hands-on

## Tricks from an old guys

Metadynamics is, at this moment, the most popular enhanced (ES) sampling technique out -there.

What is an ES technique?

\begin{equation}
    <O> = \frac{\int\ d\br \ O(\br)\ e^{-\beta U(\br)}}{\int\ d\br\ e^{-\beta U(\br)}}
\end{equation}

<center><img src='figures/high_fes.png'  style="width:50%"></center>


Let's help our simulation a little bit!

\begin{equation}
H(\br) = U(\br) + V(s,t) = U(\br) + h \sum_{t' = 0}^{T} g(s-s(t')) e^{- V(s,t')\frac{\gamma}{\gamma-1}}\\
\lim_{t \to \infty} F(s) = - \frac{\gamma}{\gamma-1}V(s,t)
\end{equation}


<center><img src='figures/meta_working.png'  style="width:50%"></center>

### Easy with PLUMED-2.0

First thing off, be sure that PLUMED can find the correct libraries with 
```bash
$ source /pat/to/plumed/sourceme.sh
```

Then lets look at the plumed.dat file that contains all the information about the biasing method we will use:

```bash
# reduced units
UNITS NATURAL
# this line define the CV (s)
d1: DISTANCE ATOMS=1,2 COMPONENTS
# This lines are usually not present in a standard simulations. Here they represent the PES that we will sample
# since is a toy model
ff: MATHEVAL ARG=d1.x VAR=x0 PERIODIC=NO FUNC=x0^2*((1.05*x0-1)*(x0+1))*200
bb: BIASVALUE ARG=ff

# This define the metadynamics calculation
mt: METAD ...
ARG=d1.x
HEIGHT=0.5 SIGMA=0.05 BIASFACTOR=30
TEMP=1.0
GRID_MIN=-1.5 GRID_MAX=1.5 GRID_BIN=400
PACE=500
...

# print the value of the CV to a file, as well as the bias of the Metadynamics
PRINT ARG=d1.x,mt.bias FILE=colvar STRIDE=500
```

For the simple toy systems we will use, PLUMED will be the molecular dynamics engine AND the Metadynamics plugin. To run a MD we use the ```pesmd``` command
```bash
plumed pesmd < input
```
which contains the information about the MD calculation

```markdown
temperature 1
tstep 0.0005
friction 1
dimension 1 
nstep 4000000
ipos 0.75
periodic false
plumed plumed.dat

```

OK Lets run the mono-dimensional example! 

**After the calculations please take a moment to plot:**
 1. The CV as a function of time (2nd col of HILLS of colvar file)
 2. The bias as a function of time (3rd col of colvar)
 3. The height of the gaussian repulsive functions (HILLS 4th col)
 4. How much bias you deposited along the CV (colvar col 3 vs col 2)

Once that the calculation is finished, reconstruct the final FES with 

```bash
plumed sum_hills --hills HILLS --mintozero --bin 200 --kt 1.0 --negbias
```

and check the profile of the FES

```bash
python ../../scripts/compare_results_reference.py --fes negativebias.dat
```

or 

```bash
python ../../scripts/compare_results_reference.py --fes negativebias.dat --ref reference/reference_fes.dat
```


### What about Free Energy Differences?

Calculating $\Delta G$ is easy with plumed, and a python script! First, create MANY fes, with a desired stride

```bash
plumed sum_hills --hills HILLS --mintozero --bin 200 --kt 1.0 --negbias --stride 100
```

This will generate many negativebias_XX.dat files. After this, use the script

```bash
python ../../scripts/calculate_fes_difference.py --prefix negativebias --mina -1 -0.5 --minb 0.5 1 --kt 1.0
```

In practice, this is done by selecting two parts of the phase space $a$ and $b$, and evaluating the probability rather than the FES. So using directly the PLUMED negativebias_XX.dat results:

\begin{equation}
P_a(t) = \int_{a-\delta}^{a+\delta}e^{-G(s,t)/kT} ds \\
P_b(t) = \int_{b-\delta}^{b+\delta}e^{-G(s,t)/kT} ds
\end{equation}
and then 

\begin{equation}
\Delta G_{ab}(t) = -kT \log \frac{P_a(t)}{P_b(t)}
\end{equation}


Using a single point as reference, is 
1. Incorrect, since there is a probability field, not "a structure"
2. Noisy, because as you decrease $\delta \to 0$ you get less and less statistics

### Thanks Fede, I will go do FES!

Or damages, most likely, because you are just grasping it atm.

<center><img src='figures/dunning-kruger.jpg'  style="width:50%"></center>


Let's redo everything with a slightly different change with mono-dimensional-bad-behaving

### It's wrong! What is off!??


Yes of course you should simulate more, but how much?

Let's start by checking what the bias and the gaussians as a function of the $s$ variable are doing!

**A simulation is converged, once the height of the repulsive gaussian is 0, EVERYWHERE**

Sing something is off,
**sometimes the convergence is drifting a bit.** Check the convergence as a function of time

### Ok fine I got it! Thanks


Yhea, but we have not finished yet.

Try the bi-dimensiona-mono-cv:
Run the calculations, check the height, recover the FES and compare with the *proj_y* fes in reference

Surprise! The height behave in a good way, but he FES does not!!!!!
WHY?

because we  neglecting a degrees of freedom

Let's see what happened if we include both of them, by doing the calculation with **bi-dimensional**

### Metadynamics in 2D

With 2D, the things get slightly more complex

Since we have two CVs, now the plumed file looks like

```markdown
mt: METAD ...
ARG=d1.x,d1.y
HEIGHT=1.0 SIGMA=0.05,0.05 BIASFACTOR=15
TEMP=1.0
GRID_MIN=-1.8,-1.8 GRID_MAX=1.8,1.8 GRID_BIN=400,400
PACE=500
...

PRINT ARG=d1.x,d1.y,mt.bias FILE=colvar STRIDE=500
```

notice that now we have two arguments, one per CV, two Sigmas, and two parameters per grid input parameter, since we now have a 2D grid

of course, if you want to recover the FES you need to explain to sum_hills that there is a 2D grid

```bash
 plumed sum_hills --hills HILLS --kt 1.0 --negbias --bin 200,200 --mintozero
 ```

###  Can I recover  1D FES with sum_hills?

sure,  you can just integrate the extra dimension out
```bash
 plumed sum_hills --hills HILLS --kt 1.0 --negbias --bin 200,200 --mintozero --idw d1.x
 ```
 
 And the result will be only the fes along the CV named d1.x
 
 Check if the mono dimensional CV converged, and check the 2D FES as well.

If you want to evaluate the $\Delta G$ then you need to specify 2 extra boundaries per minimum, of course! You can do this with 

```bash
python ../../scripts/calculate_fes_difference.py --prefix negativebias --mina -1.5 0.5 -1.5 -0.5 --minb 0.5 1.5 -0.5 1.5 --kt 1.0
```

###  And to finish the worst case scenario!

What is worst to have a neglected CV?
To have a neglected CV with a twist!

Try to check **zeta-potential-mono** and **zeta-potential** and recover the 2D as well as the 1D FES.

###  Can we obtain information on other CVs or degrees of freedom?

It is possible to recover information on other CVs or degrees of freedom!
We can evaluate what would be the weight of each configuration sampled in our Metadynamics calculations by reweigthing, so to recover the Boltzmann weight that the coonfiguation would have in an unbiased MD run.

\begin{equation}
    \hat{P}(\br,t) = P(\br)\ e^{-\beta [V(\bs(\br),t) - c(t)]}, 
\end{equation}
in which 
\begin{equation}
 e^{-\beta c(t)} = \frac{\int\ d \bs\ P(\bs)\ e^{-\beta V(\bs,t)}}{\int d \bs\ P(\bs)},
\end{equation}

the unbiased probability can thus be obtained, starting from the biased one using
\begin{equation}
    \hat{P}(\br) = \int_0^T \ dt \ e^{\beta [V(\bs,t) - c(t)]} \delta(s-s(t)), 
\end{equation}

We can evaluate the renormalization constant c(t) by using

\begin{equation}
    e^{-\beta c(t)} = \frac{\int_0^t dt'\ e^{\beta[V(\bs(t'),t')-c(t')-V(\bs(t'),t)]}}{\int_0^t dt'\ e^{\beta [V(\bs(t'),t')-c(t')]}}.
\label{eq:time_c_t}
\end{equation}

A python module that implement this schema is available in cosmo-tools in python module named as ITRE
https://github.com/cosmo-epfl/cosmo-tools.git

Check the examples folder for more complex task. For now, we will reweight the 2D calculations that we have!

When you use ITRE there are a few things you need to know:
1. You can use a stride in the calculation so that you do not need to calculate c(t) every time Usually 10--50 is ok 
2. You need the instantaneous potential, and is better to use directly the one used from PLUMED. HOWEVER the ITRE recalculated potential and the PLUMED potential should be the same! CHECK THAT!
3. The weights that you calculate, and hence c(t), need to be aligned with your trajectory! so pay attention! They should start at the same time and have the same stride!
4. PLUMED does not exactly print the HEIGHT of the gaussin when you do Well Tempered, but rather an estimate of the FES. So you need to correct it! Is easy you can just normalize the whole heights and then multiply for the heights you set at the beginning.